# **Smart Persistence Model (SPM) Implementation**



In the following i will provide **two implementations** for the benchmarck model.

**First** my own implementation, witch i used in my Bachelor thesis, this implemntation usese the PVLib libary to calculate the sun angles at any given time.

**Second** the implementation by **Yuchao Nie** witch was used in the SUNSET forcast model. (Source: https://github.com/yuhao-nie/Stanford-solar-forecasting-dataset)
 Code was Open-sourced using the MIT License:
 Copyright (c) 2022 Yuhao Nie, Xiatong Li 





## Imports and Defines

In [1]:
import numpy as np
import pandas as pd
!pip install pvlib
import pvlib
import matplotlib.pyplot as plt
import os
import pickle
from math import *
import calendar
import datetime
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 16.8 MB/s eta 0:00:00


In [2]:
# Def. functions

def root_mean_squared_error(y_true, y_pred):
    """
    Root Mean Squared Error (RMSE) custom metric function.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

def mean_absolute_error(y_true, y_pred):
    """
    Mean Absolute Error (MAE) custom metric function.
    """
    return np.mean(np.abs(y_pred - y_true))

## 1. My Own Implemantation



In [5]:
def smart_persistence_pv_forecast(labels: np.ndarray, datetime_labels: list, parameters: dict) -> list:
    """
    Calculates the Smart Persistence Predictions for given labels and timestamps.

    Parameters:
    labels (numpy.ndarray): The labels for the data.
    datetime_labels (list): The timestamps for the data.
    parameters (dict): A dictionary containing the parameters required for calculation.

    Returns:
    list: consisting of Smart Persistence Predictions.
    """

    smart_predictions = _calculate_predictions(labels, datetime_labels, parameters)
    return smart_predictions

def _calculate_predictions(labels: np.ndarray, datetime_labels: list, parameters: dict) -> list:
    times = []
    smart_predictions = []
    for i, time in enumerate(datetime_labels):
        actual_pv_output = labels[i]
        smart_pred, _, _ = _the_future_pv_output(time, actual_pv_output, parameters)
        times.append(time)
        smart_predictions.append(np.float32(smart_pred))
    return smart_predictions


def _the_future_pv_output(time: pd.Timestamp, actual_pv_output: float, parameters: dict) -> tuple:
    try:
        actual_clr_pv_output = _calculate_clr_pv_output(time, parameters)
        future_clr_pv_output = _calculate_clr_pv_output(time + pd.Timedelta(value=parameters['time_delta'], unit='minutes'), parameters)
        k_clr = actual_pv_output / actual_clr_pv_output
        future_pv_output = k_clr * future_clr_pv_output
        return future_pv_output, future_clr_pv_output, k_clr
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, None, None

def _calculate_clr_pv_output(date: pd.Timestamp, parameters: dict) -> float:
    try:
        sun_zenith_deg, sun_azimuth_deg = _calc_sun_position(date, parameters)
        sun_zenith_rad = np.radians(sun_zenith_deg)
        sun_azimuth_rad = np.radians(sun_azimuth_deg)
        panel_elevation_rad = np.radians(parameters['panel_elevation'])
        panel_azimuth_rad = np.radians(parameters['panel_azimuth'])
        pclr_t = parameters['max_solar_irradiance'] * parameters['effective_panel_area'] * (np.cos(panel_elevation_rad)* np.cos(sun_zenith_rad) + np.sin(panel_elevation_rad)* np.sin(sun_zenith_rad)* np.cos((sun_azimuth_rad-panel_azimuth_rad)))
        return pclr_t
    except Exception as e:
        print(f"An error occurred while calculating clear sky PV output: {str(e)}")
        return None

def _calc_sun_position(date: pd.Timestamp, parameters: dict) -> tuple:
    try:
        solar_position = pvlib.solarposition.get_solarposition(
                                                                date,
                                                                parameters['latitude'],
                                                                parameters['longitude'],
                                                                parameters['altitude'],
                                                                pressure=None,
                                                                method='nrel_numpy'
                                                                )
        sun_azimuth = solar_position['azimuth'].values[0]
        sun_zenith = solar_position['elevation'].values[0]
        return sun_zenith, sun_azimuth
    except Exception as e:
        print(f"An error occurred while calculating solar position: {str(e)}")
        return None, None


In [6]:
parameters = {
    'latitude': 37.42808,           # Latitude of the location in decimal degrees
    'longitude': -122.17023,        # Longitude of the location in decimal degrees
    'altitude': 23,                 # Altitude of the location in meters
    'panel_elevation': 22.5 ,       # Elevation angle of the solar PV arrays in degrees
    'panel_azimuth': 195,           # Azimuth angle of the solar PV arrays in degrees
    'max_solar_irradiance': 1000,   # Maximum solar irradiance in W/m^2
    'effective_panel_area': 24.98,  # Effective PV panel area in square meters
    'time_delta': 15                # forecast horizon in minutes
}


In [7]:
start_time = time.time()


# Function call:
predictions_implementation_1 = smart_persistence_pv_forecast(data_set["labels_test"], data_set["datetime_labels_test"], parameters)


end_time = time.time()


mae_implementation_1 = round(mean_absolute_error(data_set["labels_test"], predictions_implementation_1), 3)
execution_time_implementation_1 = round(end_time - start_time, 2)
rmse_implementation_1 = round(root_mean_squared_error(data_set["labels_test"], predictions_implementation_1), 3)

print(f"The MAE of the SPM is: {mae_implementation_1} kW")
print(f"The RMSE of the SPM is: {rmse_implementation_1} kW")
print(f"Execution time: {execution_time_implementation_1} seconds")


The MAE of the SPM is: 0.15 kW
The RMSE of the SPM is: 0.239 kW
Execution time: 162.48 seconds


## 2. Implementation by Yuchao Nie (SKIPP'D)

This implementation requires adjustments for use with a different dataset.

[SKIPP´D Dataset](https://github.com/yuhao-nie/Stanford-solar-forecasting-dataset)


In [8]:
def inputs_for_rel_op(date_and_time):
    """
    Takes a single datetime.datetime as input.
    Returns two values: 1st being day of year
    and 2nd being time of day solely in seconds-24 hr clock.
    """
    # Time correction. The center of PST is -120 W, while the logitude of the PV array is about 2 degree west of PST center

    pst_center_longitude = -120 # minus sign indicate west longitude
    panel_longitude = -122.174199 # minus sign indicate west longitude
    correction = np.abs(60/15*(panel_longitude - pst_center_longitude))
    min_correction = int(correction) # Local time delay in minutes from the PST
    sec_correction = int((correction - min_correction)*60)  # Local time delay in seconds from the PST
    if date_and_time.minute<=min_correction:
        date_and_time= date_and_time.replace(hour = date_and_time.hour-1, minute=60+date_and_time.minute-min_correction-1, second=60-sec_correction)
    else:
        date_and_time = date_and_time.replace(minute=date_and_time.minute-min_correction-1, second=60-sec_correction)

    time_of_day=date_and_time.hour * 3600 + date_and_time.minute * 60 + date_and_time.second

    # Following piece of code calculates day of year
    months=[31,28,31,30,31,30,31,31,30,31,30,31] # days in each month
    if (date_and_time.year % 4 == 0) and (date_and_time.year % 100 != 0 or date_and_time.year % 400 ==0 ) == True:
        months[1]=29 # Modification for leap year
    day_of_year=sum(months[:date_and_time.month-1])+date_and_time.day

    # Fix for daylight savings (NOTE: This doesn't work for 1st hour of each day in DST period.

    # which day of year is the 2nd Sunday of March in that year
    dst_start_day = sum(months[:2]) + calendar.monthcalendar(date_and_time.year,date_and_time.month)[1][6]
    # which day of year is the 1st Sunday of Nov in that year
    dst_end_day = sum(months[:10]) + calendar.monthcalendar(date_and_time.year,date_and_time.month)[0][6]
    if day_of_year >= dst_start_day and day_of_year < dst_end_day:
        time_of_day=time_of_day-3600

    return day_of_year, time_of_day
    #return day_of_year, time_of_day, date_and_time


def Relative_output(times,PV_ops):
    """
    Takes 2 inputs, 1st being single datetimes (e.g., Timestamp('2017-03-02 08:00:00'))
    and 2nd being single PV output at those date and times.
    Returns a corresponding theoretical and relative PV output.
    """

    # Constants
    P0=1  # Solar energy incident on Earth's surface=1 kW/m2
    A_eff=24.9842 # Effective area covered by panels in m2
    epsilon=radians(22.5) # Elevation angle of solar panel
    zeta=radians(195) # Azimuth angle of solar panel
    latitude=radians(37.424107) # Latitudinal co-ordinate of Stanford

    day_of_year, time_of_day=inputs_for_rel_op(times)
    # Calculating parameters dependent on time, day and location
    alpha=2*pi*(time_of_day-43200)/86400 # Hour angle in radians
    delta=radians(23.44*sin(radians((360/365.25)*(day_of_year-80)))); # Solar declination angle
    chi=acos(sin(delta)*sin(latitude)+cos(delta)*cos(latitude)*cos(alpha))# Zenith angle of sun
    tan_xi=sin(alpha)/(sin(latitude)*cos(alpha)-cos(latitude)*tan(delta)) # tan(Azimuth angle of sun,xi)
    if alpha>0 and tan_xi>0:
        xi=pi+atan(tan_xi)
    elif alpha>0 and tan_xi<0:
        xi=2*pi+atan(tan_xi)
    elif alpha<0 and tan_xi>0:
        xi=atan(tan_xi)
    else:
        xi=pi+atan(tan_xi)
    # Calculating theoretical output
    P_theo=P0*A_eff*(cos(epsilon)*cos(chi)+sin(epsilon)*sin(chi)*cos(xi-zeta))
    # To deal with troublesome cases
    if P_theo<0.05:
        P_theo=0.05
    if PV_ops<0.05:
        PV_ops=0.05
    Rel_PV_op=PV_ops/P_theo
    if Rel_PV_op>1.5:
        Rel_PV_op=1.5

    return Rel_PV_op, P_theo



def calculate_per_prediction(datetime_labels_test, labels_test, time_delta_minutes=15):
    """
    Calculates the per_prediction using the Relative_output function for each timestamp in datetime_labels_test.

    Parameters:
    datetime_labels_test (list): List of datetime objects representing timestamps.
    labels_test (numpy.ndarray): Array of labels representing PV outputs.
    time_delta_minutes (int, optional): The time difference in minutes between the current timestamp and the future timestamp. Defaults to 15.

    Returns:
    numpy.ndarray: Array of per_predictions.
    """
    ntimes = len(datetime_labels_test)
    per_prediction = np.zeros(ntimes)
    for i in range(ntimes):
        CSI_cur, P_theo_cur = Relative_output(datetime_labels_test[i], labels_test[i])
        CSI, P_theo_pred = Relative_output(datetime_labels_test[i] + datetime.timedelta(minutes=time_delta_minutes), labels_test[i])
        per_prediction[i] = CSI_cur * P_theo_pred
    return per_prediction




In [9]:
custom_time_delta = 15


start_time = time.time()


# Function call:
predictions_implementation_2 = calculate_per_prediction(data_set["datetime_labels_test"], data_set["labels_test"], time_delta_minutes=custom_time_delta)



end_time = time.time()


mae_implementation_2 = round(mean_absolute_error(data_set["labels_test"], predictions_implementation_2), 3)
execution_time_implementation_2 = round(end_time - start_time, 2)
rmse_implementation_2 = round(root_mean_squared_error(data_set["labels_test"], predictions_implementation_2), 3)

print(f"The MAE of the SPM is: {mae_implementation_2} kW")
print(f"The RMSE of the SPM is: {rmse_implementation_2} kW")
print(f"Execution time: {execution_time_implementation_2} seconds")


The MAE of the SPM is: 0.684 kW
The RMSE of the SPM is: 0.803 kW
Execution time: 0.61 seconds


# Comparison of the two SPM implementations

Below, I will provide some statistics comparing the two implementations so you can hopefully choose the one most suited for you.

I utilized the test set of the SKIPP'D Benchmark Dataset to acquire these metrics. ([source](https://github.com/yuhao-nie/Stanford-solar-forecasting-dataset) )


The ***main difference*** between the two implementations is how the zenith and azimuth angles of the sun are calculated.

My implementation utilizes the NREL SPA algorithm through the PVlib library, while the one from Yuchao Nie uses estimations based on empirical functions from da Rosa (Source 5).


In [10]:
# Testet with 13973 Datapoints

print("Comparison of Metrics for Two Models:")
print("-" * 50)
print(f"{'Metric':<20} {'implementation 1':<20} {'implementation 2':<20}")
print("-" * 50)
print(f"{'RMSE (kW)':<20} {rmse_implementation_1:<20} {rmse_implementation_2:<20}")
print(f"{'MAE (kW)':<20} {mae_implementation_1:<20} {mae_implementation_2:<20}")
print(f"{'Execution Time (s)':<20} {execution_time_implementation_1:<20} {execution_time_implementation_2:<20}")
print("-" * 50)
print("--> Implementation 2 is much faster, while Implementation 1 is more accurate.")


Comparison of Metrics for Two Models:
--------------------------------------------------
Metric               implementation 1     implementation 2    
--------------------------------------------------
RMSE (kW)            0.239                0.803               
MAE (kW)             0.15                 0.684               
Execution Time (s)   162.48               0.61                
--------------------------------------------------
--> Implementation 2 is much faster, while Implementation 1 is more accurate.
